In [1]:
import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

env = gym.make('CartPole-v0')

# Input and output size based on the Env
input_size = env.observation_space.shape[0] # -> 4
output_size = env.action_space.n # -> 2(left, right)

# Set learning parameters
learning_rate = .1

X = tf.placeholder(tf.float32, [None, input_size], name="input_x") # None will be 1

# First layer of weights
W1 = tf.get_variable("W1", shape=[input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
Qpred = tf.matmul(X, W1)

# We need to define the parts of the network needed for learning a policy
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

# Loss function
loss = tf.reduce_sum(tf.square(Y - Qpred))

# Learning
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Set Q-learning related parameters
dis = .99
num_episodes = 2000

# Create lists to contain total rewards and steps per episode
rList = []


[2017-08-04 13:59:43,349] Making new env: CartPole-v0


In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

sess.run(init)
for i in range(num_episodes):
    # Reset environment and get first new observation
    e = 1. / ((i / 10) + 1)
    step_count = 0
    s = env.reset()
    done = False


    # The Q-Network training
    while not done:
        step_count += 1
        x = np.reshape(s, [1, input_size])

        # Choose an action by freedily (with e change of random action) from the Q-network
        Qs = sess.run(Qpred, feed_dict = {X: x})
        if np.random.rand(1) < e:
            a = env.action_space.sample()
        else:
            a = np.argmax(Qs)

        # Get new state and reward from environment
        s1, reward, done, _ = env.step(a)
        if done:
            # Update Q, and no Qs+1, since it's terminal state
            Qs[0, a] = -100
        else:
            # Obtain the Q_s1 values by feeding the new state through our network
            x1 = np.reshape(s1, [1, input_size])
            Qs1 = sess.run(Qpred, feed_dict={X: x1})
            
            # Update Q
            Qs[0, a] = reward + dis * np.max(Qs1)

        # Train our network using target (Y) and predicted Q(Qpred) values
        sess.run(train, feed_dict={X:x, Y: Qs})
        s = s1

    rList.append(step_count)
    print("Episode: {} steps: {}".format(i, step_count))

    # If last 10's avg steps are 500, it's good enough
    if len(rList) > 10 and np.mean(rList[-10:]) > 500:
        break;


Episode: 0 steps: 14
Episode: 1 steps: 11
Episode: 2 steps: 30
Episode: 3 steps: 38
Episode: 4 steps: 19
Episode: 5 steps: 25
Episode: 6 steps: 17
Episode: 7 steps: 16
Episode: 8 steps: 30
Episode: 9 steps: 12
Episode: 10 steps: 23
Episode: 11 steps: 21
Episode: 12 steps: 100
Episode: 13 steps: 44
Episode: 14 steps: 30
Episode: 15 steps: 78
Episode: 16 steps: 33
Episode: 17 steps: 21
Episode: 18 steps: 25
Episode: 19 steps: 35
Episode: 20 steps: 23
Episode: 21 steps: 39
Episode: 22 steps: 108
Episode: 23 steps: 40
Episode: 24 steps: 34
Episode: 25 steps: 26
Episode: 26 steps: 17
Episode: 27 steps: 29
Episode: 28 steps: 34
Episode: 29 steps: 34
Episode: 30 steps: 22
Episode: 31 steps: 14
Episode: 32 steps: 25
Episode: 33 steps: 25
Episode: 34 steps: 28
Episode: 35 steps: 29
Episode: 36 steps: 27
Episode: 37 steps: 37
Episode: 38 steps: 24
Episode: 39 steps: 23
Episode: 40 steps: 25
Episode: 41 steps: 92
Episode: 42 steps: 25
Episode: 43 steps: 21
Episode: 44 steps: 13
Episode: 45 steps:

In [ ]:
# See our trained network in action
observation = env.reset()
reward_sum = 0
while True:
    env.render()
    
    x = np.reshape(observation, [1, input_size])
    Qs = sess.run(Qpred, feed_dict={X:x})
    a = np.argmax(Qs)
    
    observation, reward, done, _ = env.step(a)
    reward_sum += reward
    if done:
        print("Total score: {}".format(reward_sum))
        break